# Import libraries

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
import numpy as np
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, BatchNormalization, Dropout, Input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import plot_model
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
import pickle
import glob
from keras.utils import to_categorical

# Load processed dataset
The data we load here are already shuffled and normalized.

In [ ]:
data_directory = 'dataset/3. normalized/'
x_train = pickle.load(open(data_directory + "x_train.p", "rb"))
y_train = pickle.load(open(data_directory + "y_train.p", "rb"))
x_test = pickle.load(open(data_directory + "x_test.p", "rb"))
y_test = pickle.load(open(data_directory + "y_test.p", "rb"))
print('x_train: ' + str(x_train.shape))
print('y_train: ' + str(y_train.shape))
print('x_test: ' + str(x_test.shape))
print('y_test: ' + str(y_test.shape))

# Tunable parameters

In [ ]:
epochs = 2000
batch_size = 20
learning_rate = 0.0001
decay = 0.00001
dropout = 0.5

# Model for classification

In [ ]:
date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
root_directory = 'results/results-classify-sae-04/'
model_directory = root_directory + "model/"
tensorboard_directory = root_directory + "tensorboard/"
autoencoder_2_directory = 'results/results-autoencoder-02/model/'
autoencoder_1_path = ''
autoencoder_2_path = ''
include_raw_input_too = False
def create_model(layers):
    global date
    global autoencoder_1_path
    global autoencoder_2_path
    x = Input(shape=(x_train.shape[1],))

    y = Dense(units = layers['layer_1'], activation='relu')(x)
    y = Dropout(dropout/2)(y)

    if (layers['layer_2'] > 0):
        y = Dense(units = layers['layer_2'], activation='relu')(y)
        y = Dropout(dropout)(y)
        if (layers['layer_3'] > 0):
            y = Dense(units = layers['layer_3'], activation='relu')(y)
            y = Dropout(dropout)(y)
    
    y = Dense(units = y_train.shape[1], activation='softmax')(y)
    model = Model(x, y)
    
    # Create directory
    directory = model_directory + date + '/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Write model hyper-parameters
    df = pd.DataFrame({
        'Layer 1': layers['layer_1'],
        'Layer 2': layers['layer_2'],
        'Layer 3': layers['layer_3'],
        'Dropout 1': dropout/2,
        'Dropout 2 and 3': dropout,
        'Learning Rate': learning_rate,
        'Decay': decay,
        'Batch size': batch_size,
        'Autoencoder 1': autoencoder_1_path,
        'Autoencoder 2': autoencoder_2_path,
        'Epochs': epochs,
        'Include raw input and autoencoded input for training': include_raw_input_too
    }, index=[0])
    df.to_csv(model_directory + date + '/params.csv')
    
    # Write model summary
    file2 = open(directory + "summary.txt", "a")
    model.summary(print_fn=lambda line: file2.write(line + '\n'))
    file2.close()

    # Write model diagram
    plot_model(model, to_file=directory + 'model.png', show_shapes=True, show_layer_names=False)
    
    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate, decay=decay), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
    
    return model

# Prepare callbacks

In [ ]:
class KerasClassifierTensorBoard(KerasClassifier):
    def fit(self, x, y, **kwargs):
        global date
        date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_directory + date + '/')
        csv_logger = keras.callbacks.CSVLogger(model_directory + date + '/epochs.csv')
        model_checkpoint = keras.callbacks.ModelCheckpoint(model_directory + date + '/weights.hdf5', save_best_only=True)
        callbacks = [tensorboard_callback, csv_logger, model_checkpoint]
        return super(KerasClassifierTensorBoard, self).fit(x, y, callbacks=callbacks, verbose=0, **kwargs)

# Pass input through autoencoders

In [ ]:
autoencoder_1_path = 'results/results-autoencoder-01-02/model/2018-03-05-18-34-05/weights.hdf5'
autoencoder_2_path = 'results/results-autoencoder-02-02/model/2018-03-05-20-49-19/weights.hdf5'
weights1 = load_model(autoencoder_1_path)
weights2 = load_model(autoencoder_2_path)

# Pass input through auto encoder #1
x_train_output = weights1.predict(x_train)
x_test_output = weights1.predict(x_test)

# Pass auto encoder #1 output through auto encoder #2
x_train_output = weights2.predict(x_train_output)
x_test_output = weights2.predict(x_test_output)

# Grid search

In [ ]:
# autoencoder_2_directories = glob.glob(autoencoder_2_directory + '*')
# for autoencoder_2_path in autoencoder_2_directories:

model = KerasClassifierTensorBoard(build_fn=create_model, epochs=epochs, batch_size=batch_size)

layers = [
    {'layer_1': 10, 'layer_2': 6, 'layer_3': 4},
    {'layer_1': 4, 'layer_2': 8, 'layer_3': 16},
    {'layer_1': 4, 'layer_2': 8, 'layer_3': 0},
    {'layer_1': 16, 'layer_2': 8, 'layer_3': 4},
    {'layer_1': 32, 'layer_2': 16, 'layer_3': 8},
    {'layer_1': 32, 'layer_2': 16, 'layer_3': 0}
]
param_grid = dict(
    layers=layers
)

if include_raw_input_too:
    x_train_final = np.vstack([x_train_output, x_train])
    y_train_final = np.vstack([y_train, y_train])
else:
    x_train_final = x_train_output
    y_train_final = y_train
    
grid = GridSearchCV(estimator = model, param_grid = param_grid, error_score=0, verbose=2, cv=[(slice(None), slice(None))], n_jobs=1, fit_params=dict(validation_data=(x_test_output, y_test)))
grid_result = grid.fit(x_train_final, y_train_final)